In [1]:
from utils.df_handle import *
from math import ceil

In [2]:
import requests

In [3]:
from vars import vars

In [17]:
base_url = 'https://eoffice.merapgroup.com/eoffice/api/api/'

headers = {"Authorization": f"Bearer {vars['name_eoffice']}", "Content-Type":"application/json"}

extra_url=f'raw/data-follow?&date_start=2022-03-23&limit=500'

In [18]:
r = requests.get(base_url+extra_url, headers=headers)

data = r.json()['data']

df1=pd.DataFrame.from_records(data)

In [20]:
df1.shape

(76, 16)

In [6]:
def insert_first_time():
    #*--------------*
    #get pages
    base_url = 'https://eoffice.merapgroup.com/eoffice/api/api/'

    headers = {"Authorization": f"Bearer {vars['name_eoffice']}", "Content-Type":"application/json"}

    extra_url=f'raw/data-follow?&limit=1'
    
    print("url is: ",base_url+extra_url)
    
    r = requests.get(base_url+extra_url, headers=headers)
    
    r.json()
    
    data = r.json()['data']
    
    print("total output: " ,r.json()['total'])
    
    print("do dai list api: ",len(data))
    
    maxpage = ceil(r.json()['total']/500)
    
    pages = [x for x in range(1, maxpage+1, 1)]
    
    limit=500
    #end get pages
    #*--------------*
    for page in pages:
        base_url = 'https://eoffice.merapgroup.com/eoffice/api/api/'
        headers = {"Authorization": f"Bearer {vars['name_eoffice']}", "Content-Type":"application/json"}
        extra_url=f'raw/data-follow?limit={limit}&page={page}'
        r = requests.get(base_url+extra_url, headers=headers)
        r.json()
        data = r.json()['data']
        # len(data)
        df1=pd.DataFrame.from_records(data)
        df1.columns = ['makhdms', 'customer_phone', 'customer_name', 'follow_name',
            'follow_phone', 'gender', 'birthday', 'follow_address',
            'customer_address', 'updated_at', 'customer_role_name', 'office_code',
            'office_name', 'user_name', 'user_code', 'status']
        lst = ['makhdms', 'customer_phone', 'customer_name', 'follow_name',
            'follow_phone', 'follow_address',
            'customer_address','status','updated_at']
        df1=df1[lst]
        vc(df1, 'status')
        dk = df1['status'] != 0
        df1 = df1[dk].copy()
        # drop_cols(df1, 'status')
        print("page inserted: ", page)
        execute_values_upsert(df1, 'f_eoffice_cus_infor', pk=['follow_phone'])

In [44]:
# insert_first_time([1,2,3,4,5,6,7])